In [2]:
!pip install implicit

import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from scipy.sparse import coo_matrix
import implicit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error

     |████████████████████████████████| 18.5 MB 474 kB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
rating_file_path = '/content/drive/MyDrive/AIFFEL/data/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_df = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings_df)
ratings_df.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
movie_file_path = '/content/drive/MyDrive/AIFFEL/data/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movie_df = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movie_df.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
## Start indexing at 0
ratings_df['user_id'] = ratings_df['user_id'] - 1
ratings_df["movie_id"] = ratings_df["movie_id"] - 1
movie_df["movie_id"] = movie_df["movie_id"] - 1

## Create dictionary to easily match movie ids to titles
movie_dict = movie_df.set_index("movie_id")[["title"]].to_dict(orient="index")

## Set some ratings aside for testing

ratings_df_train, ratings_df_test = train_test_split(ratings_df, test_size=0.1, random_state=41)


In [8]:
usermovie_df = ratings_df_train.pivot(index='user_id', columns='movie_id', values='rating').reindex(range(0,ratings_df["user_id"].max()), fill_value=0).transpose().reindex(range(0,ratings_df["movie_id"].max()), fill_value=0).transpose()
usermovie_df = usermovie_df.transpose().fillna(usermovie_df.mean(axis=1)).transpose()
users = usermovie_df.index.tolist()
movies = usermovie_df.columns.tolist()
usermovie_df = usermovie_df.values



## First:
텐서플로우를 사용하여 간단한 신경망을 만듦으로써 다음과 같은 최적의 일련의 인코더 및 디코더를 찾을 수 있다.<br>

>$\overrightarrow{X}*ReLU(\overrightarrow{e_1})*\sigma(\overrightarrow{e_2})*ReLU
(\overrightarrow{d_1})*\sigma(\overrightarrow{d_2}) = \overrightarrow{X'}  \approx \overrightarrow{X}$

[Building A Collaborative Filtering Recommender System with TensorFlow](https://towardsdatascience.com/building-a-collaborative-filtering-recommender-system-with-tensorflow-82e63d27b420)

* 빈 칸은 0 대신 사용자의 평균 등급으로 채워진다.
* 첫 번째 레이어에서 사용된 ReLU 활성화 함수

In [ ]:
num_input = ratings_df["movie_id"].max()
num_hidden_1 = 20
num_hidden_2 = 7


X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

def encoder(x):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X

In [ ]:

loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, ) 
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(usermovie_df.shape[0] / batch_size)
    train_df = np.array_split(usermovie_df//5, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in train_df:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        if (i+1) % 5 == 0 : print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    train_df = np.concatenate(train_df, axis=0)

    preds = session.run(decoder_op, feed_dict={X: train_df})*5

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='rating')
    pred_data.columns = ['user_id', 'movie_id', 'rating']
    pred_data['user_id'] = pred_data['user_id'].map(lambda value: users[value])
    pred_data['movie_id'] = pred_data['movie_id'].map(lambda value: movies[value])
    
    keys = ['user_id', 'movie_id']
    index_1 = pred_data.set_index(keys).index
    index_2 = ratings_df_train.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['user_id', 'rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('user_id').head(10)

epoch: 5 Loss: 0.0085481125834333
epoch: 10 Loss: 0.008183790831068573
epoch: 15 Loss: 0.008107344850053102
epoch: 20 Loss: 0.008074943089944333
epoch: 25 Loss: 0.00805722699002471
epoch: 30 Loss: 0.008046128343097692
epoch: 35 Loss: 0.0080385553504362
epoch: 40 Loss: 0.008033075380723837
epoch: 45 Loss: 0.008028935687075
epoch: 50 Loss: 0.008025704227864396
epoch: 55 Loss: 0.008023115538255594
epoch: 60 Loss: 0.008020997856916902
epoch: 65 Loss: 0.008019235166504459
epoch: 70 Loss: 0.008017746607101587
epoch: 75 Loss: 0.00801647357036208
epoch: 80 Loss: 0.008015373302082162
epoch: 85 Loss: 0.008014413530851692
epoch: 90 Loss: 0.008013569267986471
epoch: 95 Loss: 0.008012821258933739
epoch: 100 Loss: 0.00801215430472566


# Second:
대체 최소 제곱을 통한 행렬 인수분해(GPU 최적화):

우리는 U와 M에 대한 다음 식을 바꿔가며 푼다.

>$\overrightarrow{U}\overrightarrow{M} \approx \overrightarrow{X} 
,where \\\overrightarrow{U} is\ a\ user\ (n_{users}*n_{hiddenfeatures})\ matrix, 
\\\overrightarrow{M} is\ a\ movie\ (n_{hiddenfeatures}*n_{movies})\ matrix,
$


In [ ]:
item_user_data = coo_matrix((ratings_df_train["rating"], (ratings_df_train["movie_id"],ratings_df_train["user_id"])), shape=(ratings_df_train["movie_id"].max()+1, ratings_df_train["user_id"].max()+1))
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=128, use_gpu = True)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(item_user_data)

user_items = item_user_data.T.tocsr()


In [ ]:
##Simple function to find a movie index number
def getKeysByValue(valueToFind):
    listOfKeys = list()
    listOfItems = movie_dict.items()
    for item  in listOfItems:
        if valueToFind.lower() in item[1]["title"].lower():
            listOfKeys.append((item[0],item[1]["title"]))
    return  listOfKeys

getKeysByValue("matrix")

[(2570, 'Matrix, The (1999)')]

In [ ]:
# With this approach, we are able to find similar movies
i=2570
print(movie_dict[i])
[movie_dict[x] for (x, y) in model.similar_items(i)]

{'movie': 'Matrix, The (1999)'}


[{'movie': 'Matrix, The (1999)'},
 {'movie': 'Terminator 2: Judgment Day (1991)'},
 {'movie': 'Total Recall (1990)'},
 {'movie': 'Fugitive, The (1993)'},
 {'movie': 'Terminator, The (1984)'},
 {'movie': 'Face/Off (1997)'},
 {'movie': 'Fifth Element, The (1997)'},
 {'movie': 'Twelve Monkeys (1995)'},
 {'movie': 'Jurassic Park (1993)'},
 {'movie': 'Men in Black (1997)'}]

# 결과 비교

In [ ]:
# Let's choose a random user numer
usernumber = 1234
results = pd.DataFrame()
results["watched"] = [x["title"] for x in ratings_df_train.loc[ratings_df_train['user_id'] == usernumber].sort_values(by=['rating'], ascending=False)["movie_id"].head(10).map(movie_dict)]
results["NeuralNet"] = [x["title"] for x in top_ten_ranked.loc[top_ten_ranked['user_id'] == usernumber]["movie_id"].head(10).map(movie_dict)]
results["MatrixFac"] = [movie_dict[x]["title"] for (x, y) in model.recommend(usernumber, user_items, filter_already_liked_items =True)]
results

,watched,NeuralNet,MatrixFac
0,Life Is Beautiful (La Vita � bella) (1997),Star Wars: Episode IV - A New Hope (1977),Shine (1996)
1,Titanic (1997),American Beauty (1999),Scent of a Woman (1992)
2,"Cider House Rules, The (1999)","Godfather, The (1972)",Jerry Maguire (1996)
3,Sense and Sensibility (1995),Star Wars: Episode V - The Empire Strikes Back...,"Piano, The (1993)"
4,"Joy Luck Club, The (1993)",Raiders of the Lost Ark (1981),Fried Green Tomatoes (1991)
5,Good Will Hunting (1997),"Sixth Sense, The (1999)",Awakenings (1990)
6,"Secret Garden, The (1993)",Schindler's List (1993),Schindler's List (1993)
7,"Shawshank Redemption, The (1994)","Matrix, The (1999)",Leaving Las Vegas (1995)
8,Hope Floats (1998),"Princess Bride, The (1987)","Green Mile, The (1999)"
9,"Firm, The (1993)","Usual Suspects, The (1995)","American President, The (1995)"


결과는 상당히 다르다. 

테스트 세트를 사용하여 정확도를 검사해보자.

In [ ]:
df_compare = ratings_df_test[["user_id", "movie_id", "rating"]].merge(pred_data, how="left", right_on = ["user_id", "movie_id"], left_on = ["user_id", "movie_id"], suffixes=("","_NN"))

In [ ]:

def get_als_score(user, movie):
  return model.rank_items(user, user_items,[movie])[0][1]

df_compare["Rating_MF"] = df_compare.apply(lambda x: get_als_score(int(x["user_id"]),int(x["movie_id"])), axis=1)

## Since we will simply recomend top movie, 
## we are not really interested in the actual absolute store,
## but rather by the relative scores, in that optic, it makes sense to standatdize

scaler = StandardScaler()
df_compare["Rating_standard"]= scaler.fit_transform(df_compare["rating"].values.reshape(-1,1))
df_compare["Rating_MF_standard"]= scaler.fit_transform(df_compare["Rating_MF"].values.reshape(-1,1))
df_compare["Rating_NN_standard"]= scaler.fit_transform(df_compare["Rating_NN"].values.reshape(-1,1))

df_compare=df_compare.dropna()

In [ ]:
print("Neural Network MAE: {}".format(mean_absolute_error(df_compare["Rating_standard"], df_compare["Rating_NN_standard"])))
print("Matrix Factorization MAE: {}".format(mean_absolute_error(df_compare["Rating_standard"], df_compare["Rating_MF_standard"])))

Neural Network MAE: 0.9162304069335659
Matrix Factorization MAE: 1.0391789241501572


절대오차는 둘 다 비슷하고 약 1정도??

평균적으로 예측은 실제 평가에서 표준편차 하나가 떨어져 있다는 것을 의미한다. 

아래는 원래 등급의 표준 편차(1 ~ 5 척도)도 약 1이라는 것을 볼 수 있다.
<br><br>
이 반복에서 신경망은 약간 더 나은 성능을 보였지만, 어떤 결론을 내리려면 k-폴드 CV가 필요하다.



In [ ]:
df_compare["rating"].std()

1.113562891901298